In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numba as nb

In [4]:
def e_integrand(zeta,m_e,T):
    integrand = np.zeros(len(zeta))
    for i in range(len(zeta)):
        integrand[i] = (zeta[i]**2*np.sqrt(zeta[i]**2+(m_e/T[i])**2))/(np.exp(np.sqrt(zeta[i]**2+(m_e/T[i])**2))+1)*np.exp(zeta[i])
    return integrand

x_gauss,w_gauss = np.polynomial.laguerre.laggauss(40)

This function below allows us to calcualte dQ/dt and the enegery density in neutrinos (pv) from the mass file. Takes the name of the mass file as an input and the name of the saved file you would like to be created.

In [10]:
def imp_values(filename,saved_file):
    actual_data= np.load(filename, allow_pickle=True)
    f_array = actual_data['fe']
    e_array = actual_data['e']
    temp_cm = actual_data['Tcm']
    m_s = actual_data['mass']#in MeV
    t = actual_data['time'] #inverse MeV
    a = actual_data['scalefactors']
    life = actual_data['lifetime']/(6.58*10**-25)*1/1000 #seconds need to be converted to inverse MeV
    temp = actual_data['temp'] #units of MeV
    decay = actual_data['decayrate']
    coll = actual_data['collisionrate']
    p_n = actual_data['p_n_rate']
    n_p = actual_data['n_p_rate']
    mix = actual_data['mixing']
    T_0 = temp[0]
    roww = len(temp)
    
    
    eta_0 = 6.1e-10
    fail_safe = 1
               
               
    D = (10.75/61.75)
    Rz_3 = 1.202056901178332
    n_s = D*(3*Rz_3/(2*np.pi**2))*(temp_cm)**3*np.exp(-t/life)
    ns_initial = n_s[0]
    
    
    m_pl = (1.2*10**19)*1000 #planck mass in MeV
    m_e = 0.510 #electron mass in MeV
    
    
    p_y = (np.pi**2/15)*(temp)**4
    p_vs = m_s*n_s

    results = []
    for e,f in zip(e_array,f_array):
        result = np.trapz(e**3*f,e)
        results.append(result)
        
    p_v = 6*((temp_cm)**4/(2*np.pi**2))*results #in units of MeV^4
    
    integrand_vals = e_integrand(x_gauss,m_e,temp)
    vals = np.zeros(len(integrand_vals))
    
    for i in range(len(integrand_vals)):
        vals[i] = (2*(temp[i])**4/(np.pi)**2)*integrand_vals[i]*w_gauss[i]
        
    p_e = np.sum(vals)
    p = p_e+p_y+p_v+p_vs
    
    da = np.sqrt(8*np.pi/(3*m_pl**2))*a/(p)**(-1/2)
    
    results_1 = []
    for e,dec,col in zip(e_array,decay,coll):
        result_1 = np.trapz(e**3*(dec+col),e)
        results_1.append(result_1)
    
    dQ = (m_s*n_s*a**3/life)-(temp_cm**4*a**3/(2*np.pi**2))*da*results_1 #in units of MeV^5
    
    dqdt = dQ/a**3 #converts to MeV^4/sec
    
    
   
    np.savez(saved_file, T = temp, Tcm = temp_cm, dQdt = dqdt, pn = p_n, np = n_p, rhonu = p_v, T_initial = T_0, eta_initial = eta_0, failsafe = fail_safe, ns_0 = ns_initial, vs_mass = m_s, mix_ang = mix, row = roww)

    return dqdt,p_v,
    
    

In [11]:
imp_values("mass-300-life-0.030-new.npz","correct_shortlived_vals")

(array([ 5.67898568e-16,  5.53659163e-16,  5.36917949e-16,  5.20134159e-16,
         5.03552141e-16,  4.87073281e-16,  4.70764947e-16,  4.54617542e-16,
         4.38630847e-16,  4.22882382e-16,  4.07184573e-16,  3.91724147e-16,
         3.76446166e-16,  3.61297655e-16,  3.46491659e-16,  3.31743647e-16,
         3.17231647e-16,  3.02928141e-16,  2.88866754e-16,  2.74939115e-16,
         2.61266791e-16,  2.47741453e-16,  2.34589307e-16,  2.21587104e-16,
         2.08751059e-16,  1.96295896e-16,  1.84011563e-16,  1.72017861e-16,
         1.60208992e-16,  1.48710917e-16,  1.37398426e-16,  1.26460624e-16,
         1.15678234e-16,  1.05541500e-16,  9.55236927e-17,  8.57819196e-17,
         7.70287471e-17,  6.90553190e-17,  6.11569602e-17,  5.36581694e-17,
         4.68679333e-17,  4.27967473e-17,  3.94571028e-17,  3.50425811e-17,
         3.10894343e-17,  2.74536097e-17,  2.46098377e-17,  2.25024114e-17,
         2.01061719e-17,  1.77840045e-17,  1.56456048e-17,  1.39091860e-17,
         1.2

In [5]:
actual_data= np.load("mass-300-life-0.030-new.npz", allow_pickle=True)
f_array = actual_data['fe']
e_array = actual_data['e']
temp_cm = actual_data['Tcm']
m_s = actual_data['mass']#in MeV
t = actual_data['time'] #inverse MeV
a = actual_data['scalefactors']
life = actual_data['lifetime']/(6.58*10**-25)*1/1000 #seconds need to be converted to inverse MeV
temp = actual_data['temp'] #units of MeV
decay = actual_data['decayrate']
coll = actual_data['collisionrate']
p_n = actual_data['p_n_rate']
n_p = actual_data['n_p_rate']

In [6]:
pn_reversed = p_n[::-1]
np_reversed = n_p[::-1]

In [7]:
a = np.where(pn_reversed <= 0)[0]

In [8]:
len(a)

2